In [ ]:
import pandas as pd
import numpy as np
import tqdm
import os
import glob
import cv2

## Data Splitting

In [ ]:
chexpert = pd.read_csv('/content/drive/My Drive/mimic-cxr/mimic-cxr-2.0.0-chexpert.csv')

In [ ]:
total_meta = pd.read_csv("/content/drive/My Drive/mimic-cxr/mimic-cxr-2.0.0-metadata.csv")

In [ ]:
chexpert.count()

subject_id                    227827
study_id                      227827
Atelectasis                    57666
Cardiomegaly                   66799
Consolidation                  23076
Edema                          65833
Enlarged Cardiomediastinum     21837
Fracture                        5831
Lung Lesion                     8287
Lung Opacity                   58425
No Finding                     75455
Pleural Effusion               87272
Pleural Other                   2902
Pneumonia                      59185
Pneumothorax                   53848
Support Devices                70281
dtype: int64

In [ ]:
chexpert[(chexpert['Pleural Effusion'] == 1) &(chexpert['Support Devices'] == 1)]

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
38,10001884,50712381,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0
55,10001884,56722923,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0
81,10002428,50292543,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0
82,10002428,50444997,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0
83,10002428,50862960,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227768,19997473,59017551,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0
227773,19997911,58942262,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
227794,19998562,56823087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
227799,19998843,54376373,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0


In [ ]:
chexpert.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN


In [ ]:
total_meta.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


In [ ]:
meta_merged = total_meta.merge(chexpert, left_on='study_id', right_on='study_id')

In [ ]:
meta_merged.head()

In [ ]:
meta_merged.drop(columns=['subject_id_y','PatientOrientationCodeSequence_CodeMeaning','ViewCodeSequence_CodeMeaning','ProcedureCodeSequence_CodeMeaning','StudyTime','StudyDate','Columns','Rows','PerformedProcedureStepDescription'], axis=1, inplace=True)
meta_merged = meta_merged.rename(columns={'subject_id_x': 'subject_id'})

In [ ]:
subject_with_la = meta_merged[(meta_merged['ViewPosition'] == 'LATERAL') | (meta_merged['ViewPosition'] == 'LL') ]
subject_with_la = subject_with_la['subject_id']
subject_with_la= subject_with_la.unique()

In [ ]:
subject_with_ap = meta_merged[(meta_merged['ViewPosition'] == 'AP') | (meta_merged['ViewPosition'] == 'PA')]
subject_with_ap = subject_with_ap['subject_id']
subject_with_ap = subject_with_ap.unique()

In [ ]:
subject_with_both = list(set(subject_with_ap) & set(subject_with_la))

In [ ]:
len(subject_with_both)

52576

In [ ]:
len(meta_merged['subject_id'].unique())

65379

In [ ]:
subject_with_ap_1 = np.setdiff1d(subject_with_ap,subject_with_both)

In [ ]:
np.random.shuffle(subject_with_both)
np.random.shuffle(subject_with_ap_1)

In [ ]:
print(len(subject_with_both))
print(len(subject_with_ap_1))

52576
11369


In [ ]:
train_cutpt_la = int(len(subject_with_both) * 0.7)
val_cutpt_la = train_cutpt_la + int(len(subject_with_both) * 0.15)

train_cutpt_ap = int(len(subject_with_ap) * 0.7)
val_cutpt_ap = train_cutpt_ap + int(len(subject_with_ap) * 0.15)

In [ ]:
train_la = meta_merged[meta_merged['subject_id'].isin(subject_with_both[:train_cutpt_la])]
val_la = meta_merged[meta_merged['subject_id'].isin(subject_with_both[train_cutpt_la:val_cutpt_la])]
test_la = meta_merged[meta_merged['subject_id'].isin(subject_with_both[val_cutpt_la:])]

In [ ]:
train_ap = meta_merged[meta_merged['subject_id'].isin(subject_with_ap_1[:train_cutpt_ap])]
val_ap = meta_merged[meta_merged['subject_id'].isin(subject_with_ap_1[train_cutpt_ap:val_cutpt_ap])]
test_ap = meta_merged[meta_merged['subject_id'].isin(subject_with_ap_1[val_cutpt_ap:])]

In [ ]:
train_la.head()

,dicom_id,subject_id,study_id,ViewPosition,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,LATERAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,LATERAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_la.sort_values(by=['subject_id','study_id'], inplace=True)
val_la.sort_values(by=['subject_id','study_id'], inplace=True)
test_la.sort_values(by=['subject_id','study_id'], inplace=True)

In [ ]:
train_la.to_csv("train_set_both.csv")
val_la.to_csv("val_set_both.csv")
test_la.to_csv("test_set_both.csv")

In [ ]:
train_all = pd.concat([train_ap, train_la], ignore_index=True)
val_all = pd.concat([val_ap, val_la], ignore_index=True)
test_all = pd.concat([test_ap, test_la], ignore_index=True)

In [ ]:
train_all.sort_values(by=['subject_id','study_id'], inplace=True)
val_all.sort_values(by=['subject_id','study_id'], inplace=True)
test_all.sort_values(by=['subject_id','study_id'], inplace=True)

In [ ]:
train_all.to_csv("train_set.csv")
val_all.to_csv("val_set.csv")
test_all.to_csv("test_set.csv")

## Dataloader

In [ ]:
import tensorflow as tf

In [ ]:
train = pd.read_csv("/content/drive/My Drive/AG-CNN/dataset/train_set_both.csv")
# val = pd.read_csv("/content/drive/My Drive/AG-CNN/dataset/val_set.csv")
# test = pd.read_csv("test_set.csv")

In [ ]:
def generate_path(row):
  folder = str(int(row['subject_id']))
  study_id = str(int(row['study_id']))
  dicom = row['dicom_id']
  return '/content/drive/My Drive/mimic-cxr/images512/p{}/p{}/s{}/{}.png'.format(
      folder[:2],
      folder,
      study_id,
      dicom
  )

In [ ]:
train['path'] = train.apply(lambda x : generate_path(x), axis=1)
# val['path'] = val.apply(lambda x : generate_path(x), axis=1)
# test['path'] = test.apply(lambda x : generate_path(x), axis=1)

In [ ]:
train = train[train['ViewPosition'] == 'AP']

In [ ]:
train = train[:20000]

In [ ]:
train_labels = train.iloc[:, 5:19]

In [ ]:
# train_labels.fillna(0, inplace=True)

# train_labels.replace(-1, 0, inplace=True)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train['path'],np.array(train_labels), train['study_id']))

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_merge['study_id'],train_merge['frontal'], train_merge['side'], np.array(train_labels)))

In [ ]:
def decode_img(img):
  img = tf.image.decode_png(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  return tf.image.resize(img, (224,224))

def process_path(path, label, sid):
  img = tf.io.read_file(path)
  img = decode_img(img)
  return img, label, sid

def process_path_mul(sid, front, side, label):
  img_front = tf.io.read_file(front)
  img_front = decode_img(img_front)

  img_side = tf.io.read_file(side)
  img_side = decode_img(img_side)

  return img_front, img_side, label, sid

In [ ]:
train_im = train_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    .apply(tf.data.experimental.ignore_errors()).cache('/content/train_cache')

In [ ]:
train_im = train_ds.map(process_path_mul, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    .apply(tf.data.experimental.ignore_errors()).cache('/content/train_cache')

### Create TFRecord

In [ ]:
len(train)

75080

In [ ]:
BS = 64
# TR_SHARDS = 40
TR_SHARDS = 10
GCS_OUTPUT = '/content/drive/My Drive/AG-CNN/tfrecord_train_ap/' 

In [ ]:
import math

tr_size = len(train)
tr_shard_size = math.ceil(1.0 * tr_size / TR_SHARDS)

In [ ]:
tr_shard_size

2000

In [ ]:
def recompress_image(image, label, sid):
  image = image * 255
  image = tf.cast(image, tf.uint8)
  image = tf.image.encode_jpeg(image, optimize_size=True, chroma_downsampling=False)
  return image, label, sid

In [ ]:
def recompress_image_mul(image_front,image_side, label, sid):
  image_front = image_front * 255
  image_front = tf.cast(image_front, tf.uint8)
  image_front = tf.image.encode_jpeg(image_front, optimize_size=True, chroma_downsampling=False)

  image_side = image_side * 255
  image_side = tf.cast(image_side, tf.uint8)
  image_side = tf.image.encode_jpeg(image_side, optimize_size=True, chroma_downsampling=False)
  return image_front, image_side, label, sid

In [ ]:
train_imc = train_im.map(recompress_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_imc = train_imc.batch(tr_shard_size).prefetch(1)

In [ ]:
# val_imc = val_im.map(recompress_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# val_imc = val_imc.batch(val_shard_size).prefetch(1)

In [ ]:
# Three types of data can be stored in TFRecords: bytestrings, integers and floats
# They are always stored as lists, a single data element will be a list of size 1

def _bytestring_feature(list_of_bytestrings):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def _int_feature(list_of_ints): # int64
  return tf.train.Feature(int64_list=tf.train.Int64List(value=list_of_ints))

def _float_feature(list_of_floats): # float32
  return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_floats))
  

def to_tfrecord(tfrec_filewriter, img, label, sid):  
  
  feature = {
      "image": _bytestring_feature([img]), 
      "label":         _float_feature(label.reshape(-1)), 
      "study_id": _int_feature([sid])
    }
  return tf.train.Example(features=tf.train.Features(feature=feature))


In [ ]:
print("Writing TFRecords")
for shard, (image, label, sid) in tqdm.tqdm(enumerate(train_imc)):
  # batch size used as shard size here
  shard_size = image.numpy().shape[0]
  # good practice to have the number of records in the filename
  filename = GCS_OUTPUT + "{:02d}-{}.tfrec".format(shard, shard_size)
  if os.path.exists(filename): 
    continue
  
  with tf.io.TFRecordWriter(filename) as out_file:
    for i in tqdm.tqdm(range(shard_size)):
      example = to_tfrecord(out_file,
                            image.numpy()[i],
                            label.numpy()[i],
                            sid.numpy()[i])
      out_file.write(example.SerializeToString())
    print("Wrote file {} containing {} records".format(filename, shard_size))

0it [00:00, ?it/s]

Writing TFRecords



100%|██████████| 2000/2000 [01:15<00:00, 26.65it/s]
1it [25:47, 1547.48s/it]

Wrote file /content/drive/My Drive/AG-CNN/tfrecord_train_ap/00-2000.tfrec containing 2000 records



100%|██████████| 2000/2000 [01:17<00:00, 25.93it/s]
2it [46:47, 1461.28s/it]

...

Wrote file /content/drive/My Drive/AG-CNN/tfrecord_train_ap/05-2000.tfrec containing 2000 records



100%|██████████| 2000/2000 [01:51<00:00, 18.01it/s]
7it [2:56:36, 1565.68s/it]

Wrote file /content/drive/My Drive/AG-CNN/tfrecord_train_ap/06-2000.tfrec containing 2000 records



100%|██████████| 2000/2000 [02:07<00:00, 15.67it/s]
8it [3:29:10, 1682.07s/it]

Wrote file /content/drive/My Drive/AG-CNN/tfrecord_train_ap/07-2000.tfrec containing 2000 records


In [ ]:
def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "label": tf.io.VarLenFeature(tf.float32)        # a certain number of floats0
        "size":  tf.io.FixedLenFeature([2], tf.int64),  # two integers
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding
    
    image = tf.image.decode_jpeg(example['image'], channels=3)
    label  = tf.reshape(tf.sparse.to_dense(example['label']), [-1, 14])
    height = example['size'][0]
    width  = example['size'][1]
    image = tf.reshape(image, [height, width, 3])
    return image, label

In [ ]:
# read from TFRecords. For optimal performance, read from multiple
# TFRecord files at once and set the option experimental_deterministic = False
# to allow order-altering optimizations.

option_no_order = tf.data.Options()
option_no_order.experimental_deterministic = False

filenames = tf.io.gfile.glob(GCS_OUTPUT + "*.tfrec")
train_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
train_dsr = train_dsr.with_options(option_no_order)
train_dsr = train_dsr.map(read_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dsr = train_dsr.shuffle(300)